# Literary Analysis: Comparing Nonfiction and Fiction through Topic Modeling and Sentiment Analysis 

### ADS 509 Final Project
##### Team 3: Claire Bentzen, Tara Dehdari, Logan Van Dine

##### Introduction

In this project, we will conduct a comparative analysis of two significant literary works: "Pride and Prejudice" by Jane Austen (fiction) and "A Vindication of the Rights of Woman" by Mary Wollstonecraft (nonfiction). Both texts engage deeply with themes of gender, society, and individual rights, making them ideal for exploring the differences in language, themes, and sentiment between fiction and nonfiction.

Using text mining techniques, we will analyze how each genre approaches these themes, examining the stylistic and rhetorical differences that characterize fiction versus nonfiction. Our analysis will involve data cleaning, tokenization, and the application of descriptive statistics, sentiment analysis, and topic modeling. By comparing these works, we aim to uncover the unique ways in which each genre communicates similar ideas, providing insights into the broader distinctions between fiction and nonfiction writing.


### Imports

In [1]:
import requests
import os
import re
import pandas as pd
import nltk

from bs4 import BeautifulSoup  
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter

c:\Users\lvand\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Scraping

This portion scrapes and saves the full text of Pride and Prejudice and A Vindication of the Rights of Woman from Project Gutenberg. It ensures the save directory exists, extracts text from the HTML, saves it to .txt files, and verifies that the files were created successfully, showing a preview of the content.

In [3]:
# Define the URLs for the books
## fiction
url_pride_prej = 'https://www.gutenberg.org/cache/epub/1342/pg1342-images.html'
url_awakening = 'https://www.gutenberg.org/cache/epub/160/pg160-images.html'
url_north_south = 'https://www.gutenberg.org/cache/epub/4276/pg4276-images.html'
url_wuthering_heights = 'https://www.gutenberg.org/cache/epub/768/pg768-images.html'
## nonfiction
url_vin_of_women = 'https://www.gutenberg.org/cache/epub/3420/pg3420-images.html'
url_enfranchisement = 'https://www.gutenberg.org/cache/epub/73404/pg73404-images.html'
url_on_liberty = 'https://www.gutenberg.org/cache/epub/34901/pg34901-images.html'
url_subjection_women = 'https://www.gutenberg.org/cache/epub/27083/pg27083-images.html'

# Define the directory to save the files
data_dir = './data'

# Ensure the directory exists
os.makedirs(data_dir, exist_ok=True)

# Function to scrape and save books
def scrape_and_save_book(url, file_name):
    # Send a GET request to the URL
    response = requests.get(url)
    response.raise_for_status()  # Check that the request was successful
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract all text from <p> tags
    paragraphs = soup.find_all(['p', 'h1', 'h2', 'h3'])
    book_text = '\n'.join([para.get_text() for para in paragraphs if para.get_text().strip()])

    
    # Save the extracted text to a file
    file_path = os.path.join(data_dir, file_name)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(book_text)
    
    print(f"Text from '{file_name}' scraped and saved.")
    
    # Check if the file is saved and contains content
    if os.path.exists(file_path):
        print(f"File '{file_path}' has been created successfully.")
        # Check the first few lines of the file
        with open(file_path, 'r', encoding='utf-8') as file:
            preview = file.read(100)  # Read the first 100 characters
            print("File content preview:\n")
            print(preview)
    else:
        print(f"Failed to create the file '{file_path}'.")

# List of Fiction book tuples
fiction_books = [
    (url_pride_prej, 'pride_and_prejudice.txt'),
    (url_awakening, 'the_awakening.txt'),
    (url_north_south, 'north_and_south.txt'),
    (url_wuthering_heights, 'wuthering_heights.txt')
]

# Loop over the list and scrape/save each book
for url, filename in fiction_books:
    scrape_and_save_book(url, filename)

# List of Nonfiction book tuples
nonfiction_books = [
    (url_vin_of_women, 'vindication_of_rights_of_woman.txt'),
    (url_enfranchisement, 'the_enfranchisement_of_women.txt'),
    (url_on_liberty, 'on_liberty.txt'),
    (url_subjection_women, 'the_subjection_of_women.txt')
]

# Loop over the list and scrape/save each book
for url, filename in nonfiction_books:
    scrape_and_save_book(url, filename)

Text from 'pride_and_prejudice.txt' scraped and saved.
File './data\pride_and_prejudice.txt' has been created successfully.
File content preview:

The Project Gutenberg eBook of Pride and Prejudice
Title: Pride and Prejudice
Author: Jane Austen
Re
Text from 'the_awakening.txt' scraped and saved.
File './data\the_awakening.txt' has been created successfully.
File content preview:

The Project Gutenberg eBook of The Awakening, and Selected Short Stories
Title: The Awakening, and S
Text from 'north_and_south.txt' scraped and saved.
File './data\north_and_south.txt' has been created successfully.
File content preview:

The Project Gutenberg eBook of North and South
Title: North and South
Author: Elizabeth Cleghorn Gas
Text from 'wuthering_heights.txt' scraped and saved.
File './data\wuthering_heights.txt' has been created successfully.
File content preview:

The Project Gutenberg eBook of Wuthering Heights
Title: Wuthering Heights
Author: Emily Brontë
Relea
Text from 'vindication_of_rights

### Data Cleaning and Tokenization

This section converts the raw text into a dataframe format that includes information about the books.

In [8]:
# Initialize empty dataframe
books = pd.DataFrame(columns=['Title', 'Author', 'Release_Date', 'Updated_Date', 'Language', 'Credits', 'Text', 'Genre'])

# Function to convert text to DataFrame
def convert_to_df(file_name, genre):
    # Establish file path
    file_path = os.path.join(data_dir, file_name)
    
    # Open contents of file
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            
            # Extract relevant sections with error handling for missing fields
            title = re.search(r'Title:\s*(.*)', content)
            title = title.group(1) if title else 'Unknown'
            
            author = re.search(r'Author:\s*(.*)', content)
            author = author.group(1) if author else 'Unknown'
            
            release_date = re.search(r'Release date:\s*(.*)\[eBook', content)
            release_date = release_date.group(1).strip() if release_date else 'Unknown'
            
            updated_date = re.search(r'Most recently updated:\s*(.*)', content)
            updated_date = updated_date.group(1) if updated_date else 'Not available'
            
            language = re.search(r'Language:\s*(.*)', content)
            language = language.group(1) if language else 'Unknown'
            
            credits = re.search(r'Credits:\s*(.*)', content)
            credits = credits.group(1) if credits else 'Not available'
            
            # Extract book text
            match = re.search(r'Credits:.*?\n(.*)', content, re.DOTALL)
            book_text = match.group(1).strip() if match else 'No text available'
            
            # Dictionary for book information
            book_info = {
                'Title': title,
                'Author': author,
                'Release_Date': release_date,
                'Updated_Date': updated_date,
                'Language': language,
                'Credits': credits,
                'Text': book_text,
                'Genre': genre  
            }
            
            # Add data to books DataFrame
            books.loc[len(books)] = book_info
            
    else:
        print(f"The file '{file_name}' does not exist.")



##### Call on Dataframe Function with both texts

In [9]:
# Create list/loop to call on dataframe function
# Combined list of  tuples for both Fiction and Nonfiction
books_list = [
    ('pride_and_prejudice.txt', 'Fiction'),
    ('the_awakening.txt', 'Fiction'),
    ('north_and_south.txt', 'Fiction'),
    ('wuthering_heights.txt', 'Fiction'),
    ('vindication_of_rights_of_woman.txt', 'Nonfiction'),
    ('the_enfranchisement_of_women.txt', 'Nonfiction'),
    ('on_liberty.txt', 'Nonfiction'),
    ('the_subjection_of_women.txt', 'Nonfiction')
]

# Loop over the combined list and convert to DataFrame
for filename, genre in books_list:
    convert_to_df(filename, genre)

In [11]:
# Display the books DataFrame to check the loaded texts
books.head()

,Title,Author,Release_Date,Updated_Date,Language,Credits,Text,Genre
0,Pride and Prejudice,Jane Austen,"June 1, 1998","June 17, 2024",English,Chuck Greif and the Online Distributed Proofre...,"PREFACE.\nList of Illustrations.\nChapter: I.,...",Fiction
1,"The Awakening, and Selected Short Stories",Kate Chopin,"March 11, 2006","February 28, 2021",English,Judith Boss and David Widger,The Awakeningand Selected Short Stories\nby Ka...,Fiction
2,North and South,Elizabeth Cleghorn Gaskell,"July 1, 2003","February 8, 2024",English,Produced by Chuck Greif and the Online Distrib...,NORTH AND SOUTH.\n“SHE LAY CURLED UPON THE SOF...,Fiction
3,Wuthering Heights,Emily Brontë,"December 1, 1996","January 18, 2022",English,David Price,Wuthering Heights\nby Emily Brontë\nCHAPTER I\...,Fiction
4,A Vindication of the Rights of Woman,Mary Wollstonecraft,"September 1, 2002","January 8, 2021",English,"This etext was produced by Amy E Zelmer, Col C...",This etext was produced by\nAmy E Zelmer <a.z...,Nonfiction


This section cleans and tokenizes the Text column with the following steps:
1. Cast to lowercase
2. Remove punctuation
3. Tokenize
4. Remove stopwords

In [12]:
# Punctuation
punctuation = set(punctuation) 

# Removes punctuation
def remove_punctuation(text, punct_set=punctuation): 
    
    return("".join([ch for ch in text if ch not in punct_set]))

# Stopwords
sw = stopwords.words("english")

# Removes stopwords
def remove_stop(tokens):
    
    tokens = [word for word in tokens if word not in sw]
    
    return(tokens)
 

# Tokenize the text
def tokenize(text):     
    
    return text.split()

# Applies the pipeline
def pipeline(text): 
    
    text = str.lower(text)
    text = remove_punctuation(text)
    tokens = tokenize(text)
    tokens = remove_stop(tokens)
    
    return(' '.join(tokens))

In [13]:
# Converts Text column to string
books['Text'] = books['Text'].astype(str)

# Cleans Text
books['Cleaned_Text'] = books['Text'].apply(pipeline)

# Tokenizes Text
books['Tokens'] = books['Cleaned_Text'].apply(tokenize)

In [40]:
books

,Title,Author,Release_Date,Updated_Date,Language,Credits,Text,Genre,Cleaned_Text,Tokens
0,Pride and Prejudice,Jane Austen,"June 1, 1998","June 17, 2024",English,Chuck Greif and the Online Distributed Proofre...,"PREFACE.\nList of Illustrations.\nChapter: I.,...",Fiction,preface list illustrations chapter ii iii iv v...,"[preface, list, illustrations, chapter, ii, ii..."
1,"The Awakening, and Selected Short Stories",Kate Chopin,"March 11, 2006","February 28, 2021",English,Judith Boss and David Widger,The Awakeningand Selected Short Stories\nby Ka...,Fiction,awakeningand selected short stories kate chopi...,"[awakeningand, selected, short, stories, kate,..."
2,North and South,Elizabeth Cleghorn Gaskell,"July 1, 2003","February 8, 2024",English,Produced by Chuck Greif and the Online Distrib...,NORTH AND SOUTH.\n“SHE LAY CURLED UPON THE SOF...,Fiction,north south “she lay curled upon sofa back dra...,"[north, south, “she, lay, curled, upon, sofa, ..."
3,Wuthering Heights,Emily Brontë,"December 1, 1996","January 18, 2022",English,David Price,Wuthering Heights\nby Emily Brontë\nCHAPTER I\...,Fiction,wuthering heights emily brontë chapter 1801—i ...,"[wuthering, heights, emily, brontë, chapter, 1..."
4,A Vindication of the Rights of Woman,Mary Wollstonecraft,"September 1, 2002","January 8, 2021",English,"This etext was produced by Amy E Zelmer, Col C...",This etext was produced by\nAmy E Zelmer <a.z...,Nonfiction,etext produced amy e zelmer azelmercqueduau co...,"[etext, produced, amy, e, zelmer, azelmercqued..."
5,Enfranchisement of women,Harriet Hardy Taylor Mill,"April 16, 2024",Not available,English,Claudine Corbasson and the Online Distributed ...,To the reader\nFootnotes\n1\nENFRANCHISEMENT O...,Nonfiction,reader footnotes 1 enfranchisement women mrs j...,"[reader, footnotes, 1, enfranchisement, women,..."
6,On Liberty,John Stuart Mill,"January 10, 2011","August 12, 2019",English,"Produced by Curtis Weyant, Martin Pettit and t...",Distributed Proofreading Team at http://www.pg...,Nonfiction,distributed proofreading team httpwwwpgdpnet p...,"[distributed, proofreading, team, httpwwwpgdpn..."
7,The Subjection of Women,John Stuart Mill,"October 28, 2008","January 25, 2021",English,Produced by Michael Roe and the Online Distrib...,Proofreading Team at https://www.pgdp.net (Thi...,Nonfiction,proofreading team httpswwwpgdpnet file produce...,"[proofreading, team, httpswwwpgdpnet, file, pr..."


### Exploratory Data Analysis

##### Descriptive Statistics

In [18]:
# Function to pull descriptive statistics from clean, tokenized text
def descriptive_stats(tokens, title, num_tokens=5, verbose=True):
    if verbose:
        print(f"Descriptive statistics for '{title}':")
        print(f"There are {len(tokens)} tokens in the text.")
        print(f"There are {len(set(tokens))} unique tokens in the text.")
        print(f"There are {len(''.join(tokens))} characters in the text.")
        print(f"The lexical diversity is {len(set(tokens))/len(tokens):.3f} in the text.")

        counts = Counter(tokens)

        if num_tokens > 0 : 
            print(counts.most_common(num_tokens))
            print('\n') # add spacing for cleaner output
        
    return([len(tokens),
            len(set(tokens)),
            len("".join(tokens)),
            len(set(tokens))/len(tokens)])

In [19]:
# Loop through each title and run descriptive statistics
for index, row in books.iterrows():
    title = row['Title']
    tokens = row['Tokens']
    
    # Call the descriptive_stats function for each book
    descriptive_stats(tokens, title, num_tokens=10)

Descriptive statistics for 'Pride and Prejudice':
There are 59460 tokens in the text.
There are 8684 unique tokens in the text.
There are 373076 characters in the text.
The lexical diversity is 0.146 in the text.
[('mr', 780), ('elizabeth', 580), ('could', 527), ('would', 480), ('said', 380), ('darcy', 356), ('mrs', 346), ('much', 327), ('must', 312), ('miss', 301)]


Descriptive statistics for 'The Awakening, and Selected Short Stories':
There are 32102 tokens in the text.
There are 7732 unique tokens in the text.
There are 191489 characters in the text.
The lexical diversity is 0.241 in the text.
[('edna', 281), ('upon', 259), ('one', 253), ('would', 194), ('little', 186), ('pontellier', 171), ('like', 162), ('said', 160), ('mrs', 159), ('robert', 139)]


Descriptive statistics for 'North and South':
There are 88256 tokens in the text.
There are 13345 unique tokens in the text.
There are 508748 characters in the text.
The lexical diversity is 0.151 in the text.
[('margaret', 1186), (